# Ground state of Diatomic Hydrogen using XACC and VQE

In [1]:
import pyxacc as xacc
import pyxaccvqe as vqe
from openfermion.hamiltonians import MolecularData
from openfermionpsi4 import run_psi4
from openfermion.transforms import get_fermion_operator

## Define the Molecule

In [13]:
bond_length = .7474
geometry = [('H', (0,0,0)),('H',(0,0,bond_length))]
basis = 'sto-3g'
mdata = MolecularData(geometry,basis,1)

## Generate the Hamiltonian coefficients and get FCI energy

In [14]:
molecule = run_psi4(mdata, run_scf=1, run_fci=1)
print('FCI Energy = ', molecule.fci_energy)
print('Fermionic Hamiltonian = \n', molecule.get_molecular_hamiltonian())

FCI Energy =  -1.1371757101731694
Fermionic Hamiltonian = 
 () 0.7080240949826064
((0, 1), (0, 0)) -1.248846801817026
((1, 1), (1, 0)) -1.248846801817026
((2, 1), (2, 0)) -0.4796778151607899
((3, 1), (3, 0)) -0.4796778151607899
((0, 1), (1, 1), (1, 0), (0, 0)) 0.33667197218932576
((0, 1), (1, 1), (3, 0), (2, 0)) 0.0908126658307406
((0, 1), (2, 1), (0, 0), (2, 0)) 0.09081266583074038
((0, 1), (2, 1), (2, 0), (0, 0)) 0.331213646878486
((0, 1), (3, 1), (1, 0), (2, 0)) 0.09081266583074038
((0, 1), (3, 1), (3, 0), (0, 0)) 0.331213646878486
((1, 1), (0, 1), (0, 0), (1, 0)) 0.33667197218932576
((1, 1), (0, 1), (2, 0), (3, 0)) 0.0908126658307406
((1, 1), (2, 1), (0, 0), (3, 0)) 0.09081266583074038
((1, 1), (2, 1), (2, 0), (1, 0)) 0.331213646878486
((1, 1), (3, 1), (1, 0), (3, 0)) 0.09081266583074038
((1, 1), (3, 1), (3, 0), (1, 0)) 0.331213646878486
((2, 1), (0, 1), (0, 0), (2, 0)) 0.331213646878486
((2, 1), (0, 1), (2, 0), (0, 0)) 0.09081266583074052
((2, 1), (1, 1), (1, 0), (2, 0)) 0.3312136

## Map to OpenFermion FermionOperator

In [15]:
fop = get_fermion_operator(molecule.get_molecular_hamiltonian()) 
print(fop)

0.7080240949826064 [] +
-1.248846801817026 [0^ 0] +
-1.248846801817026 [1^ 1] +
-0.4796778151607899 [2^ 2] +
-0.4796778151607899 [3^ 3] +
0.33667197218932576 [0^ 1^ 1 0] +
0.0908126658307406 [0^ 1^ 3 2] +
0.09081266583074038 [0^ 2^ 0 2] +
0.331213646878486 [0^ 2^ 2 0] +
0.09081266583074038 [0^ 3^ 1 2] +
0.331213646878486 [0^ 3^ 3 0] +
0.33667197218932576 [1^ 0^ 0 1] +
0.0908126658307406 [1^ 0^ 2 3] +
0.09081266583074038 [1^ 2^ 0 3] +
0.331213646878486 [1^ 2^ 2 1] +
0.09081266583074038 [1^ 3^ 1 3] +
0.331213646878486 [1^ 3^ 3 1] +
0.331213646878486 [2^ 0^ 0 2] +
0.09081266583074052 [2^ 0^ 2 0] +
0.331213646878486 [2^ 1^ 1 2] +
0.09081266583074052 [2^ 1^ 3 0] +
0.09081266583074048 [2^ 3^ 1 0] +
0.34814578469185886 [2^ 3^ 3 2] +
0.331213646878486 [3^ 0^ 0 3] +
0.09081266583074052 [3^ 0^ 2 1] +
0.331213646878486 [3^ 1^ 1 3] +
0.09081266583074052 [3^ 1^ 3 1] +
0.09081266583074048 [3^ 2^ 0 1] +
0.34814578469185886 [3^ 2^ 2 3]


## Compile to XACC Hamiltonian for Accelerator Execution

In [17]:
xaccOp = vqe.compile(fop)
print(xaccOp)

(0.174073,0) Z2 Z3 + (0.120201,0) Z1 Z3 + (0.165607,0) Z0 Z3 + (0.120201,0) Z0 Z2 + (0.0454063,0) X0 Y1 Y2 X3 + (-0.0454063,0) Y0 Y1 X2 X3 + (0.0454063,0) Y0 X1 X2 Y3 + (0.165607,0) Z1 Z2 + (-0.0454063,0) X0 X1 Y2 Y3 + (0.168336,0) Z0 Z1 + (-0.220042,0) Z3 + (0.170281,0) Z1 + (-0.220042,0) Z2 + (0.170281,0) Z0 + (-0.10648,0)


## Execute VQE with Nelder Mead Optimizer and UCCSD Ansatz

In [18]:
result = vqe.execute(xaccOp, **{'n-electrons':2,'task':'vqe'})
print('VQE Energy = ', result.energy)
print('VQE Optimized Angles = ', result.angles)

VQE Energy =  -1.1371813393211314
VQE Optimized Angles =  [-1.5704786   0.72821066]
